# 1. 네비인사이드:블랙박스 포럼 crawling

### (1) 블랙박스 불평/불만 게시판 url 주소 리스트 저장

1) import 모듈

In [24]:
import requests
import re
from bs4 import BeautifulSoup

In [25]:
base_url = 'http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&select_arrange=headnum&desc=asc&category=103\
&sn=off&ss=on&sc=on&keyword=&sn1=&divpage=15&page='

2) url링크 주소를 담을 리스트 생성 (url_link)

In [26]:
url_link = []

In [27]:
for i in range(1,4):
    try:
        print i
        res = requests.post(base_url+str(i))
        soup = BeautifulSoup(res.text, 'html5lib')
        td_list = soup.find_all('td', attrs = {'class' : 'ctl_sj'})
        for td in td_list:
            url_link.append("http://www.navinside.com/bbs/{}".format( td.a['href']))
    except:
        break  

1
2
3


3) 리스트에 크롤링된 url 링크 주소 확인

In [28]:
len(url_link)

122

In [29]:
url_link[121]

'http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=3&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=22959'

### (2) 불평/불만 내용 리스트 저장

1) import 모듈

In [60]:
from selenium import webdriver

2) 블랙박스에 대한 불평/불만 내용을 담을 리스트 생성 (url_content)

In [61]:
url_title = []

In [62]:
url_content = []

In [63]:
chromedriver = './chromedriver'
driver = webdriver.Chrome(chromedriver)
for i in url_link:
    #print i
    driver.get(i)
    try:
        #mid_contents > div > table.xe_viewTb > tbody > tr:nth-child(1) > td.xe_viewSj
        black_title = driver.find_element_by_css_selector("tbody > tr > td.xe_viewSj")
        #print black_title.text
        url_title.append(black_title.text.encode('utf-8'))
        black_content = driver.find_element_by_css_selector("div.xe_viewMemo")
        #black_content = black_content.text
        #black_content = black_content.strip()
        url_content.append(black_content.text.encode('utf-8'))
    except Exception as e:
        print (e , i)
driver.quit()

(NoSuchElementException(), 'http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=2&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=29152')


3) 리스트에 크롤링된 불평/불만 내용 확인

In [64]:
print len(url_title)

121


In [65]:
print len(url_content)

121


In [66]:
print url_title[1]

아이로드T8 설치후기


# 2.정규표현식을 사용하여 크롤링 데이터 전처리

In [104]:
new_list = []

In [105]:
len(url_content)

121

In [106]:
for string in url_content:
    replaced = re.sub(r'click="" ( [\w.-]+) />', r'', string)
    #print(string)
    replaced = re.sub(r'click="" src="http://www.navinside.com/bbs/data/multi_new2011/[\d.-]+/[\w.-]+.jpg" />', r'', string)
    #replaced = re.sub(r'click="" src="http://www.navinside.com/bbs/data/multi_new2011/[a-zA-Z_0-9]+.jpg" />', r'', string)
    replaced = replaced = re.sub(r'http://[a-zA-Z_0-9_._/_?_=_&]+', r'', replaced)
    replaced = re.sub(r'-[_-]+', r'', replaced)
    replaced = re.sub(r'=[_=]+', r'', replaced)
    replaced = re.sub(r'"', r'', replaced)
    replaced = re.sub(r"'", r'', replaced)
    #print replaced
    new_list.append(replaced)

In [107]:
print len(new_list)

121


# 3. AWS 서버 mysql db에  import

In [121]:
server = 'server_ip'
db_id = 'root'
db_pwd = 'password'
db_db = 'db_name'

In [122]:
import MySQLdb
db = MySQLdb.connect(server, db_id, db_pwd, db_db)

cursor = db.cursor()

In [123]:
type(url_link[34])

str

In [124]:
type(url_title[34])

str

In [125]:
type(url_content[34])

str

In [126]:
for i in range(len(new_list)):
    try:
        #print url_link[i]
        query = """INSERT INTO blackbox VALUES \
        ({}, '네비인사이드' , '{}' ,'{}','{}','negative');""".format(i , url_link[i], url_title[i], new_list[i])
        cursor.execute(query)
    except Exception as e:
        db.rollback()
        print (e)

db.commit()
cursor.close()

    # disconnect from server
db.close()

http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=1&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=77569
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=1&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=77508
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=1&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=76851
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=1&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=76483
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=1&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=75880
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=1&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=74037
http://www.navinside.com/bbs/zboard.php?id=forum_ins

http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=2&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=43201
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=2&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=43157
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=2&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=42987
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=2&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=42690
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=2&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=41284
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=2&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=39904
http://www.navinside.com/bbs/zboard.php?id=forum_ins

http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=3&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=25525
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=3&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=25528
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=3&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=25524
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=3&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=25047
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=3&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=25013
http://www.navinside.com/bbs/zboard.php?id=forum_ins_blackbox&page=3&sn1=&divpage=15&category=103&sn=off&ss=on&sc=on&select_arrange=headnum&desc=asc&no=24883
http://www.navinside.com/bbs/zboard.php?id=forum_ins